# Final submission

- less gates, less noises: optimze a shallow circuit with naive VQE + CHC-S + COBYLA
- unbalanced better than balanced: a bit random disturb on ansatz `zeros` init point
- less pulses, less noises and faster: apply instrution optimization at QASM level

In [1]:
from pprint import pprint as pp
import utils; utils.log = lambda *args, **kwargs: None   # mute debug log
from run_eval import *

# the common settings for all
args = Namespace()
args.S = 'Montreal'           # system model
args.N = ''                   # noise model
args.simulator = 'automatic'  # Aer simulator
args.device = 'CPU'           # Aer device
args.shots = 6000             # fixed as contest
args.H = str(HAM_FILE)        # load ham from 'OHhamiltonian.txt'
args.thresh = 0.0             # ham trim thresh
args.exp_dp = None            # leave it blank so far
args.seed = None              # leave it blank so far
print('args:')
pp(vars(args))
print()

# the required seeds to average
print('required seeds:', seeds)

args:
{'H': 'd:\\Desktop\\Workspace\\@Contest\\ICCAD 2023 Drug Discovery '
      'Challenge\\QC-Contest-Demo\\Hamiltonian\\OHhamiltonian.txt',
 'N': '',
 'S': 'Montreal',
 'device': 'CPU',
 'exp_dp': None,
 'seed': None,
 'shots': 6000,
 'simulator': 'automatic',
 'thresh': 0.0}

required seeds: [20, 21, 30, 33, 36, 42, 43, 55, 67, 170]


⚪ ref_val

In [2]:
# the ref_val (ground truth)
ref_val = solver_const(None, None)
print('ref_val:', ref_val)

ref_val: -74.3871462681872


In [3]:
# helper func
def eval_with_seed(args, seed:int, run_pulse:bool=True) -> Tuple[float, int]:
  args.seed = seed
  seed_everything(seed)
  return eval(args, is_run_pulse=run_pulse)

mean = lambda x: sum(x) / len(x) if len(x) else 0.0
error = lambda x, y: abs(x - y)
error_rate = lambda x, y: error(x, y) / abs(y)

def eval_qsam_over_seeds(args):
  # load pretrained qasm
  args.qsam_fp = args.exp_dp / 'ansatz_t.qsam'
  args.layout_fp = args.exp_dp / 'ansatz_t.layout'

  # run over different seeds
  energies, duration = [], None
  for i, seed in enumerate(seeds, 1):
    ene, dur = eval_with_seed(args, seed, run_pulse=i==1)
    print(f'[{i}/{len(seeds)}] seed: {seed}, ene: {ene}')
    energies.append(ene)
    if dur is not None: duration = dur
  enegry = mean(energies)

  # average results
  print('=' * 42)
  print('enegry:', enegry)
  print('duration:', duration)
  print('error:', error(enegry, ref_val))
  print(f'error rate: {error_rate(enegry, ref_val):%}')

⚪ cairo

In [4]:
args.N = 'cairo'
args.exp_dp = Path('log') / 'final_cairo'

eval_qsam_over_seeds(args)

[1/10] seed: 20, ene: -69.11139417296809
[2/10] seed: 21, ene: -68.97114970791017
[3/10] seed: 30, ene: -69.16073187307605
[4/10] seed: 33, ene: -69.00671669325858
[5/10] seed: 36, ene: -68.90630251684358
[6/10] seed: 42, ene: -68.97104080428488
[7/10] seed: 43, ene: -69.07062703016864
[8/10] seed: 55, ene: -69.07335774085372
[9/10] seed: 67, ene: -69.11696980427236
[10/10] seed: 170, ene: -68.9312336639572
enegry: -69.03195240075932
duration: 54720
error: 5.355193867427872
error rate: 7.199085%


⚪ kolkata

In [5]:
args.N = 'kolkata'
args.exp_dp = Path('log') / 'final_kolkata'

eval_qsam_over_seeds(args)

[1/10] seed: 20, ene: -70.2328895154707
[2/10] seed: 21, ene: -70.37434193042415
[3/10] seed: 30, ene: -70.21706384716643
[4/10] seed: 33, ene: -70.36939881875236
[5/10] seed: 36, ene: -70.02593568133052
[6/10] seed: 42, ene: -69.93228657505138
[7/10] seed: 43, ene: -70.16115412639735
[8/10] seed: 55, ene: -70.24382127930383
[9/10] seed: 67, ene: -70.25425025852559
[10/10] seed: 170, ene: -70.22571716980066
enegry: -70.2036859202223
duration: 54720
error: 4.183460347964896
error rate: 5.623902%


⚪ montreal

In [6]:
args.N = 'montreal'
args.exp_dp = Path('log') / 'final_montreal'

eval_qsam_over_seeds(args)

[1/10] seed: 20, ene: -69.02157813146442
[2/10] seed: 21, ene: -69.30033624299611
[3/10] seed: 30, ene: -69.01286361552518
[4/10] seed: 33, ene: -69.09466674766409
[5/10] seed: 36, ene: -68.8731209171242
[6/10] seed: 42, ene: -68.90867789020665
[7/10] seed: 43, ene: -68.88077757703296
[8/10] seed: 55, ene: -69.0950297324026
[9/10] seed: 67, ene: -69.027826363135
[10/10] seed: 170, ene: -69.08864707984091
enegry: -69.03035242973921
duration: 54720
error: 5.3567938384479845
error rate: 7.201236%


⚠ noiseless (for debug use, not for final submission)

In [ ]:
args.N = ''
args.exp_dp = Path('log') / 'test'

eval_qsam_over_seeds(args)